## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


# SyriaTel Customer Churn Classification — Phase 3 Project

**Goal:** Build and evaluate classification models that predict whether a SyriaTel customer will churn, and translate model performance into business recommendations.

**Primary stakeholder:** SyriaTel Customer Retention / Revenue team  
**Business question:** *Which customers are at risk of churning soon so we can intervene early and reduce lost revenue?*

> **Primary metric:** Recall for the churn (positive) class — missing a true churner (false negative) is costly.

In [4]:
import numpy as np
import pandas as pd
import zipfile

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    recall_score,
    precision_score,
    f1_score,
    roc_auc_score,
    RocCurveDisplay,
    ConfusionMatrixDisplay
)

import matplotlib.pyplot as plt

RANDOM_STATE = 42

## 1. Data Loading

This notebook expects a **CSV** file for the SyriaTel churn dataset.

- If your file is named something like `syria_tel_churn.csv`, place it in the same folder as this notebook and update the path below.
- The target column is expected to be named **`churn`** (case-insensitive works in the helper below).

In [17]:
import pandas as pd

df = pd.read_csv(
    "zippedData/bigml_59c28831336c6604c8008a5a.csv",
    compression="zip"
)

df.head()




FileNotFoundError: [Errno 2] No such file or directory: 'zippedData/bigml_59c28831336c6604c8008a5a.csv'

In [6]:

df = pd.read_csv("zippedData/syria_tel_churn.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'zippedData/syria_tel_churn.csv'